# NBA Game Outcome

In [34]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression 
from sklearn import metrics
import pickle
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [27]:
games_df = pd.concat(map
                     (pd.read_csv, 
                          ['data/game_details/nba_game_detailsSeason=2021-22.csv', 
                           'data/game_details/nba_game_detailsSeason=2020-21.csv', 
                           'data/game_details/nba_game_detailsSeason=2019-20.csv',
                           'data/game_details/nba_game_detailsSeason=2018-19.csv',
                           'data/game_details/nba_game_detailsSeason=2017-18.csv',
                           'data/game_details/nba_game_detailsSeason=2016-17.csv',
                           'data/game_details/nba_game_detailsSeason=2015-16.csv',
                           'data/game_details/nba_game_detailsSeason=2014-15.csv',
                           'data/game_details/nba_game_detailsSeason=2013-14.csv',
                           'data/game_details/nba_game_detailsSeason=2012-13.csv']), ignore_index=True)

In [28]:
y = []
for i in range(0,3000):#len(games_df)):
# Check Matchup to see which team is Home
    if "@" in games_df["MATCHUP"][i]:
        away_team = games_df["TEAM_ABBREVIATION"]
        home_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(0)
        else:
            y.append(1)
         
    else:
        home_team = games_df["TEAM_ABBREVIATION"]
        away_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(1)
        else:
            y.append(0)
last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)


C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
(last_10h[last_10h["WL"]=="W"])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
3027,22020,1610612742,DAL,Dallas Mavericks,22000673,2021-03-24,DAL @ MIN,W,240,48,...,39,53,27,5,6,10,22,128,20,1
3075,22020,1610612742,DAL,Dallas Mavericks,22000652,2021-03-21,DAL @ POR,W,240,48,...,44,52,23,4,3,11,17,132,40,1
3140,22020,1610612742,DAL,Dallas Mavericks,22000625,2021-03-17,DAL vs. LAC,W,240,40,...,37,43,14,7,7,18,16,105,16,1
3211,22020,1610612742,DAL,Dallas Mavericks,22000592,2021-03-13,DAL @ DEN,W,240,42,...,37,48,26,5,3,8,19,116,13,1
3257,22020,1610612742,DAL,Dallas Mavericks,22000567,2021-03-10,DAL vs. SAS,W,240,43,...,38,51,29,3,1,15,19,115,11,1
3285,22020,1610612742,DAL,Dallas Mavericks,22000549,2021-03-03,DAL vs. OKC,W,240,32,...,46,53,18,8,3,15,13,87,9,1
3314,22020,1610612742,DAL,Dallas Mavericks,22000529,2021-03-01,DAL @ ORL,W,240,47,...,39,46,29,6,6,8,12,130,6,1


In [83]:
# Creating rows for dataframe
trans_df = pd.DataFrame()
y = []
for i in range(0,6000):#len(games_df)):
    # Check Matchup to see which team is Home
    if "@" in games_df["MATCHUP"][i]:
        away_team = games_df["TEAM_ABBREVIATION"][i]
        home_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(0)
        else:
            y.append(1)
         
    else:
        home_team = games_df["TEAM_ABBREVIATION"][i]
        away_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(1)
        else:
            y.append(0)
    # row list for df
    row = []
    
    #Last 10 games df for both teams
    last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
    last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
    
    win_rateH = 0
    win_rateA = 0
    
    needed_features = ["FGA", "FG_PCT", "FG3_PCT", "DREB", "REB", "AST"]
    
    for feature in needed_features:
        row.append(round((sum(last_10h[feature])/10),2))
        row.append(round((sum(last_10a[feature])/10),2))
    
    row.append(round((len(last_10h[last_10h["WL"]=="W"])/10),2))
    row.append(round((len(last_10a[last_10a["WL"]=="W"])/10),2))
    
    trans_df = trans_df.append(pd.DataFrame([row]), ignore_index=True)

trans_df=trans_df.set_axis(["FGA_HOME","FGA_AWAY", "FG_PCT_HOME", "FG_PCT_AWAY", "FG3_PCT_HOME", "FG3_PCT_AWAY", 
                               "DREB_HOME", "DREB_AWAY", "REB_HOME", "REB_AWAY", "AST_HOME", "AST_AWAY", "WIN_RATE_HOME", "WIN_RATE_AWAY"], axis=1)
        
trans_df["WIN"] = y

trans_df.head()

C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY,WIN
0,85.2,88.9,0.49,0.44,0.34,0.39,34.5,33.6,44.3,44.1,29.1,24.3,0.6,0.4,1
1,85.2,88.9,0.49,0.44,0.34,0.39,34.5,33.6,44.3,44.1,29.1,24.3,0.6,0.4,1
2,91.5,87.1,0.48,0.48,0.41,0.34,37.7,31.3,49.4,39.3,25.3,23.7,0.8,0.3,1
3,91.5,87.1,0.48,0.48,0.41,0.34,37.7,31.3,49.4,39.3,25.3,23.7,0.8,0.3,1
4,86.8,90.8,0.44,0.47,0.38,0.35,36.2,31.1,48.6,41.9,22.0,25.9,0.5,0.6,0


# Read the Data and Perform Basic Data Cleaning

In [4]:
columns = ["HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "PTS_home", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home", "AST_home", "REB_home", "PTS_away", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away", "AST_away", "REB_away"]

target = ["HOME_TEAM_WINS"]

In [31]:
"""# Load the data from AWS - REVIEW WITH TEAM
file_path ="https://YOUR-BUCKET-NAME.s3.amazonaws.com/INSERTLINK"
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null comumns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Convert the target column values to win/loss based on their values
x = {'1': 'Win'}   
df = df.replace(x)

x = {'0': 'Loss'}   
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

#create column with FGA difference between teams
trans_df["FGA_DIFF"]=trans_df["FGA_HOME"]-trans_df["FGA_AWAY"]
trans_df.drop(columns=["FGA_HOME","FGA_AWAY"], inplace=True)
trans_df.head()

#Create column with FG percent ratio between teams
trans_df["FG_PCT_RATIO"]=trans_df["FG_PCT_HOME"]/trans_df["FG_PCT_AWAY"]
trans_df.drop(columns=["FG_PCT_HOME", "FG_PCT_AWAY"], inplace=True)

#Create column with 3-point FG percent ratio between teams
trans_df["FG3_PCT_RATIO"] = trans_df["FG3_PCT_HOME"]/trans_df["FG3_PCT_AWAY"]
trans_df.drop(columns=["FG3_PCT_HOME", "FG3_PCT_AWAY"], inplace = True)

#Create column with Defensive rebound difference between teams
trans_df["DREB_DIFF"]=trans_df["DREB_HOME"]-trans_df["DREB_AWAY"]
trans_df.drop(columns=["DREB_HOME", "DREB_AWAY"], inplace =True)

#Create column with rebound difference between teams
trans_df["REB_DIFF"]=trans_df["REB_HOME"]-trans_df["REB_AWAY"]
trans_df.drop(columns=["REB_HOME","REB_AWAY"], inplace=True)

#Create column with assist difference between teams
trans_df["AST_DIFF"]=trans_df["AST_HOME"]-trans_df["AST_AWAY"]
trans_df.drop(columns=["AST_HOME","AST_AWAY"], inplace=True)

win=trans_df["WIN"]
trans_df.head()

win=trans_df["WIN"]
trans_df.drop(columns=["WIN"], inplace=True)
trans_df["WIN"]=win
trans_df.head()
"""

,FGA_DIFF,FG_PCT_RATIO,FG3_PCT_RATIO,DREB_DIFF,REB_DIFF,AST_DIFF,WIN
0,-0.60,1.042553,0.971429,3.25,3.30,2.95,1
1,-1.00,1.021739,0.897436,-0.80,-0.70,0.85,1
2,-0.45,0.959184,0.972222,0.25,1.70,2.35,1
3,2.85,1.042553,1.114286,2.10,2.50,-0.50,1
4,1.70,0.914894,1.000000,1.75,4.95,-3.60,0


# Split the Data into Training and Testing

In [84]:
#Creating input and output data
trans_df["WIN"] = y
y = trans_df["WIN"]
X_df = trans_df.drop(["WIN"], axis=1)

In [85]:
# Check the balance of our target values
y.value_counts()

1    3256
0    2744
Name: WIN, dtype: int64

In [86]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=42, stratify=y)
X_train

,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY
4593,84.1,89.5,0.47,0.46,0.31,0.37,31.8,35.3,40.6,44.6,26.9,23.7,0.2,0.6
1821,85.0,89.6,0.48,0.45,0.38,0.35,32.9,31.9,42.2,43.6,23.8,20.6,0.7,0.5
4321,86.4,86.4,0.44,0.47,0.30,0.39,32.5,36.6,43.1,45.8,21.9,22.2,0.4,0.6
3671,88.5,86.1,0.46,0.47,0.39,0.41,31.7,36.4,42.5,47.9,28.4,24.2,0.4,0.9
317,87.6,90.8,0.48,0.48,0.35,0.38,36.9,33.3,47.6,44.3,27.0,25.0,0.8,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772,89.7,89.8,0.49,0.46,0.38,0.33,33.4,33.4,46.0,42.5,28.3,22.4,0.8,0.2
4131,92.2,89.1,0.47,0.44,0.40,0.34,38.6,31.6,49.4,41.6,25.6,24.2,0.4,0.1
5333,89.2,90.2,0.46,0.49,0.33,0.43,36.3,39.4,48.8,48.0,22.1,25.2,0.8,0.5
4295,84.1,86.6,0.46,0.49,0.35,0.35,33.9,34.2,42.3,43.8,26.0,26.2,0.3,0.4


In [87]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Balanced Random Forest Classifier

In [88]:
# Resample the training data with the BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [89]:
# Calculated the balanced accuracy score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8864907844499681

In [90]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[596,  90],
       [ 78, 736]], dtype=int64)

In [91]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.87      0.90      0.88      0.89      0.78       686
          1       0.89      0.90      0.87      0.90      0.89      0.79       814

avg / total       0.89      0.89      0.88      0.89      0.89      0.79      1500



In [14]:
# List the features sorted in descending order by feature importance
feature_names = X_df.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.08945070816714402, 'AST_AWAY'),
 (0.08454745223154003, 'REB_AWAY'),
 (0.08364036892947889, 'REB_HOME'),
 (0.08264865832181231, 'DREB_HOME'),
 (0.08243720313762762, 'FGA_HOME'),
 (0.08164072128066058, 'AST_HOME'),
 (0.08139592945621177, 'FGA_AWAY'),
 (0.07855077624995423, 'DREB_AWAY'),
 (0.06483977926590531, 'WIN_RATE_HOME'),
 (0.06317882796804182, 'WIN_RATE_AWAY'),
 (0.05545627560217806, 'FG3_PCT_HOME'),
 (0.05260913796581507, 'FG3_PCT_AWAY'),
 (0.05014614847430904, 'FG_PCT_HOME'),
 (0.049458012949321215, 'FG_PCT_AWAY')]

In [41]:
X_train_scaled

array([[-0.00578913,  1.07482227,  1.28043993, ...,  1.65834116,
        -0.97128893,  0.49667005],
       [ 0.35512013,  0.40956311,  0.81977638, ...,  1.27029174,
         0.45552288,  1.4509891 ],
       [-2.10562483,  1.10808523, -0.56221427, ..., -1.96345339,
         0.93112682,  0.49667005],
       ...,
       [ 0.05983074,  0.60914086, -0.10155072, ...,  1.48587475,
        -0.97128893,  0.97382958],
       [-0.72760765, -0.28895901, -1.48354137, ..., -0.36813913,
        -0.97128893,  1.4509891 ],
       [ 0.05983074, -0.05611831, -0.10155072, ..., -0.97177155,
         0.45552288,  0.97382958]])

In [16]:
X_train.head(1)

,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY
2669,88.1,91.6,0.49,0.49,0.34,0.39,33.6,36.3,43.2,48.0,27.9,28.4,0.3,0.6


In [92]:
random_forest.predict_proba(X_train.head(5))

array([[0.94, 0.06],
       [0.79, 0.21],
       [0.95, 0.05],
       [0.92, 0.08],
       [0.08, 0.92]])

## Logistic Regression model

In [93]:
# create complex Logistic Regression with max_iter=131 
log_model = LogisticRegression(max_iter=131, verbose=2, random_state=42)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)
print(metrics.accuracy_score(y_test, y_pred_log))

0.5966666666666667


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [94]:
# save the model to disk
filename = 'finalized_log_model.sav'
pickle.dump(log_model, open(filename, 'wb'))

In [95]:
# save the model to disk
filename = 'finalized_RF_model.sav'
pickle.dump(random_forest, open(filename, 'wb'))

In [44]:
result = random_forest.score(X_test, y_test)
print(result)

0.876


In [48]:
prob = random_forest.predict_proba(X_train.head(1))

In [50]:
prob[0][0]

0.99

In [110]:
home_team = "MIL"
away_team = "ATL"
# generate the table from your two teams
trans_df = pd.DataFrame()

# row list for df
row = []
        
#Last 10 games df for both teams
last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
        
win_rateH = 0
win_rateA = 0
        
needed_features = ["FGA", "FG_PCT", "FG3_PCT", "DREB", "REB", "AST"]
        
for feature in needed_features:
    row.append(round((sum(last_10h[feature])/10),2))
    row.append(round((sum(last_10a[feature])/10),2))
        
row.append(round((len(last_10h[last_10h["WL"]=="W"])/10),2))
row.append(round((len(last_10a[last_10a["WL"]=="W"])/10),2))
        
trans_df = trans_df.append(pd.DataFrame([row]), ignore_index=True)

trans_df=trans_df.set_axis(["FGA_HOME","FGA_AWAY", "FG_PCT_HOME", "FG_PCT_AWAY", "FG3_PCT_HOME", "FG3_PCT_AWAY", 
                                "DREB_HOME", "DREB_AWAY", "REB_HOME", "REB_AWAY", "AST_HOME", "AST_AWAY", "WIN_RATE_HOME", "WIN_RATE_AWAY"], axis=1)
            

trans_df.head()

C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY
0,91.7,88.9,0.47,0.45,0.33,0.3,41.4,32.3,52.9,43.3,24.1,22.1,0.9,0.2


In [111]:
        #Creating input data
        X_df = trans_df

        # Create a StandardScaler instances
        scaler = StandardScaler()

        # Fit the StandardScaler
        X_scaler = scaler.fit(X_df)
        
        #Scale the input
        X_scaled = X_scaler.transform(X_df)

        # pass in the data into the model
        prob = random_forest.predict_proba(X_df)
        print(X_df)
        prob

   FGA_HOME  FGA_AWAY  FG_PCT_HOME  FG_PCT_AWAY  FG3_PCT_HOME  FG3_PCT_AWAY  \
0      91.7      88.9         0.47         0.45          0.33           0.3   

   DREB_HOME  DREB_AWAY  REB_HOME  REB_AWAY  AST_HOME  AST_AWAY  \
0       41.4       32.3      52.9      43.3      24.1      22.1   

   WIN_RATE_HOME  WIN_RATE_AWAY  
0            0.9            0.2  


array([[0.22, 0.78]])